**2-site variational matrix product state(varMPS)**

This task considers a 1D spin (ising - cluster) model with OBC:
$$
H  = -\sum_j (Z_{j-1} Y_j Z_{j+1} +Z_j Z_{j+1} + X_j),
$$
where $X$, $Y$ and $Z$ are Pauli matrices. 
For $3$-body terms, $(j-1, j, j+1)=(0,1,2),(1,2,3),\cdots,(N-2,N-1,N)$.
For $2$-body terms, $(j,j+1) = (0,1),(1,2),\cdots,(N-1,N)$.
For single-body terms, $j = 0,1,\cdots, N$.

We can easily write down the MPOs of this model, using finite-state-automata method:
$$
M = \begin{pmatrix}
\mathrm{Id}&-Z&0&-X\\
0&0&Y&Z\\
0&0&0&Z\\
0&0&0&\mathrm{Id}
\end{pmatrix}
$$
We will calculation of ground-state energy and the magnetization per site $\langle X_i\rangle$ and $\langle Z_i\rangle$ of this model, using varMPS(2-site), varMPS(1-site) and exact diagonalization.

In [18]:
import numpy as np
import numpy.linalg as LA
import scipy.sparse.linalg as LAs
import Sub180221 as Sub
import math,copy


Ns = 10
Dp = 2
Ds = 6

#----------------------------------------------------------------
"""
Mi =
S0 -Z 0 X
0  0  Y Z
0  0  0 Z
0  0  0 I

"""
#get the mpo for isingcluster using the subroutine Sub.SpinOper
def GetMpo_IsingCluster_Obc(Dp):
    S0,Sp,Sm,Sz,Sx,Sy = Sub.SpinOper(Dp)

    Dmpo = 4
    Mpo = np.zeros((Dmpo,Dp,Dmpo,Dp))+0j

    Mpo[0,:,0,:] = S0
    Mpo[0,:,1,:] = -2.0*Sz
    Mpo[0,:,3,:] = -2.0*Sx
    Mpo[1,:,2,:] = 2.0*Sy
    Mpo[1,:,3,:] = 2.0*Sz
    Mpo[2,:,3,:] = 2.0*Sz
    Mpo[3,:,3,:] = S0

    return Mpo

#Initialize the right-canonical mps, using LQ decomposition
def InitMps(Ns,Dp,Ds):
    T = [None]*Ns
    for i in range(Ns):
        Dl = min(Dp**i,Dp**(Ns-i),Ds)
        Dr = min(Dp**(i+1),Dp**(Ns-1-i),Ds)
        T[i] = np.random.rand(Dl,Dp,Dr)

    U = np.eye(np.shape(T[-1])[-1])
    for i in range(Ns-1,0,-1):
        U,T[i] = Sub.Mps_LQP(T[i],U)

    return T

#Initialize the right-environmental tensors by contracting the mpos and component tensors T
def InitH(Mpo,T):
    Ns = len(T)
    Dmpo = np.shape(Mpo)[0]

    HL = [None]*Ns
    HR = [None]*Ns

    HL[0] = np.zeros((1,Dmpo,1))
    HL[0][0,0,0] = 1.0
    HR[-1] = np.zeros((1,Dmpo,1))
    HR[-1][0,-1,0] = 1.0

    for i in range(Ns-1,0,-1):
        HR[i-1] = Sub.NCon([HR[i],T[i],Mpo,np.conj(T[i])],[[1,3,5],[-1,2,1],[-2,2,3,4],[-3,4,5]])

    return HL,HR

#Optimizting T site using eigenvalue decomposition, returning the optimal T and the energy on this site
def OptTSite(Mpo,HL,HR,T,Method=0):
    DT = np.shape(T)
    Dl = np.prod(DT)

    if Method == 0:
        A = Sub.NCon([HL,Mpo,HR],[[-1,1,-4],[1,-5,2,-2],[-6,2,-3]])
        A = Sub.Group(A,[[0,1,2],[3,4,5]])+0j
        Eig,V = LAs.eigsh(A,k=1,which='SA')
        T = np.reshape(V,DT)

    if Method == 1:
        def UpdateV(V):
            V = np.reshape(V,DT)
            V = Sub.NCon([HL,V,Mpo,HR],[[-1,3,1],[1,2,4],[3,2,5,-2],[4,5,-3]])
            V = np.reshape(V,[Dl])
            return V

        V0 = np.reshape(T,[Dl])

        MV = LAs.LinearOperator((Dl,Dl),matvec=UpdateV)
        Eig,V = LAs.eigsh(MV,k=1,which='SA',v0=V0)
        T = np.reshape(V,DT)
        Eig = np.real(Eig)

    return T,Eig

#sweep back and forth to successively optimize each T until the energy convergence is reached
def OptT(Mpo,HL,HR,T):
    Ns = len(T)
    Eng0 = np.zeros(Ns)
    Eng1 = np.zeros(Ns)

    for r in range(100):

        for i in range(Ns-1):
            T[i],Eng1[i] = OptTSite(Mpo,HL[i],HR[i],T[i],Method=1)
            #print(i,Eng1[i])
            T[i],U = Sub.Mps_QR0P(T[i])
            
            #updating HL[i+1] using HL[i] and the optimized T[i]
            HL[i+1] = Sub.NCon([HL[i],np.conj(T[i]),Mpo,T[i]],[[1,3,5],[1,2,-1],[3,4,-2,2],[5,4,-3]])
            
            #-U-T[i+1]-  -->  -T[i+1]- (new)
            T[i+1] = np.tensordot(U,T[i+1],(1,0))

        for i in range(Ns-1,0,-1):
            T[i],Eng1[i] = OptTSite(Mpo,HL[i],HR[i],T[i],Method=1)
            #print(i,Eng1[i])
            U,T[i] = Sub.Mps_LQ0P(T[i])
            
            #updating HR[i-1] using HR[i] and the optimized T[i]
            HR[i-1] = Sub.NCon([HR[i],T[i],Mpo,np.conj(T[i])],[[1,3,5],[-1,2,1],[-2,2,3,4],[-3,4,5]])
            
            #-T[i-1]-U-  --> -T[i-1]- (new)
            T[i-1] = np.tensordot(T[i-1],U,(2,0))

        if abs(Eng1[1]-Eng0[1]) < 1.0e-7:
            break
        Eng0 = copy.copy(Eng1)
        
    # print the output energy (per-site & average)
    print("(1) Energy per Site:{}".format(Eng1/float(Ns)))
    print("(1) Energy average:{}".format(np.mean(Eng1/float(Ns))))
    return T

#define Pauli-Z, Pauli-X and identity matrices
pZ = np.array([[1,0],[0,-1]])
pX = np.array([[0,1],[1,0]])
pY = np.array([[0,-1j],[1j,0]])
eye = np.array([[1,0],[0,1]])

#optimize each 2-site from left to right
def OptT2SiteL(Mpo, HL, HR, T1, T2):
    DT1 = np.shape(T1)
    DT2 = np.shape(T2)
    
    # -T1-T2- -> -T1T2-
    #  |   |       ||
    Contract12 = Sub.NCon([T1,T2],[[-1,-2,1],[1,-3,-4]])
    
    # get the shape of -T1T2-
    #                  ||
    D = np.shape(Contract12)
   
    # get the effective Hamiltonian by contracting environmental tensors and the mpos
    A = Sub.NCon([HL,Mpo,Mpo,HR],[[-1,1,-5],[1,-6,2,-2],[2,-7,3,-3],[-8,3,-4]])
    A = Sub.Group(A, [[0,1,2,3],[4,5,6,7]])
    
    # apply eigenvalue decomposition to A and reshape V to - V -
    #                                                       ||
    Eig,V = LAs.eigsh(A, k=1, which='SA')
    V = np.reshape(V, D)
    
    # reshape V to a matrix and apply SVD
    V = Sub.Group(V, [[0,1],[2,3]])
    UL, Sing, VR = LA.svd(V,full_matrices=False)
   
    # left singular vector matrix as new T1, then contract schmidt matrix and right singular vector matrix as new T2
    # = V =  -->  = U - S - V = --> -U-S-V-  -->  -[U-S]-V-  --> -T1-T2- (new)
    #                                |   |          |    |        |   |
    
    #truncate bond dimension
    max_vdim = min(Sing.shape[0],Ds)
    Sing_trunc = np.diag(Sing[:max_vdim])
    UL = UL[:,:max_vdim].reshape(V.shape[0],max_vdim)
    VR = VR[:max_vdim,:].reshape(max_vdim,V.shape[1])
    T1 = np.reshape(UL, DT1[:2]+(int(np.prod(UL.shape)/np.prod(DT1[:2])),))
    T2 = np.reshape(Sing_trunc @ VR, (int(np.prod((Sing_trunc @ VR).shape)/np.prod(DT2[1:])),)+DT2[1:])
    
    return T1, T2, Eig

#optimize each 2-site from right to left
def OptT2SiteR(Mpo, HL, HR, T1, T2):
    DT1 = np.shape(T1)
    DT2 = np.shape(T2)
    
    # -T1-T2- -> -T1T2-
    #  |   |       ||
    Contract12 = Sub.NCon([T1,T2],[[-1,-2,1],[1,-3,-4]])
    D = np.shape(Contract12)
    
    # get the effective Hamiltonian by contracting environmental tensors and the mpos
    A = Sub.NCon([HL,Mpo,Mpo,HR],[[-1,1,-5],[1,-6,2,-2],[2,-7,3,-3],[-8,3,-4]])
    A = Sub.Group(A, [[0,1,2,3],[4,5,6,7]])

    
    # apply eigenvalue decomposition to A and reshape V to - V -
    #                                                       ||
    Eig,V = LAs.eigsh(A, k=1, which='SA')
    V = np.reshape(V, D)
    
    # reshape V to a matrix and apply SVD
    V = Sub.Group(V, [[0,1],[2,3]])
    UL, Sing, VR = LA.svd(V,full_matrices=False)
    
    # contract left singular vector matrix and singular vector matrix as new T1, then right singular vector matrix as new T2.
    # = V =  -->  = U - S - V = --> -U-S-V-  -->  -U-[S-V]-  --> -T1-T2- (new)
    #                                |   |        |    |          |   |
    
    #truncate bond dimension
    max_vdim = min(Sing.shape[0],Ds)
    Sing_trunc = np.diag(Sing[:max_vdim])
    UL = UL[:,:max_vdim].reshape(V.shape[0],max_vdim)
    VR = VR[:max_vdim,:].reshape(max_vdim,V.shape[1])
    T1 = np.reshape(UL @ Sing_trunc, DT1[:2]+(int(np.prod((UL @ Sing_trunc).shape)/np.prod(DT1[:2])),))
    T2 = np.reshape(VR, (int(np.prod(VR.shape)/np.prod(DT2[1:])),)+DT2[1:])
    
    return T1, T2, Eig

#sweep back and forth to successively optimize each T until the energy convergence is reached
def OptT2(Mpo, HL, HR, T):
    Ns = len(T)
    Eng0 = np.zeros(Ns)
    Eng1 = np.zeros(Ns)
    
    for r in range(100):
        for i in range(0,Ns-1):
            T[i],T[i+1],Eig = OptT2SiteL(Mpo,HL[i],HR[i+1],T[i],T[i+1])
            Eng1[i] = Eig
            
            #update HL[i+1] using HL[i] and T[i]. Notice that there is no need to update HL[i+2]
            HL[i+1] = Sub.NCon([HL[i],np.conj(T[i]),Mpo,T[i]],[[1,3,5],[1,2,-1],[3,4,-2,2],[5,4,-3]])
            
        for i in range(Ns-2,-1,-1):
            T[i],T[i+1],Eig = OptT2SiteR(Mpo,HL[i],HR[i+1],T[i],T[i+1])
            Eng1[i+1] = Eig
            
            #update HR[i] using HR[i+1] and T[i+1]. Notice that there is no need to update HR[i-1]
            HR[i] = Sub.NCon([HR[i+1],T[i+1],Mpo,np.conj(T[i+1])],[[1,3,5],[-1,2,1],[-2,2,3,4],[-3,4,5]])
            
        #print(Eng1)
        if abs(Eng1[1]-Eng0[1]) < 1.0e-7:
            break
        Eng0 = copy.copy(Eng1)
        
    # print the output energy (per-site & average)
    print("(1) Energy per Site:{}".format(Eng1/float(Ns)))
    print("(1) Energy average:{}".format(np.mean(Eng1/float(Ns))))
    return T

def Get_magnet(T,op):
    Ns = len(T)
    
    # create an empty list to record the <O> on each site
    per_site = []
    S = copy.copy(T)
    
    # use LQ decomposition to get the right-canonical form of T
    U = np.eye(np.shape(S[-1])[-1])
    for i in range(Ns-1,0,-1):
        U,S[i] = Sub.Mps_LQP(S[i],U)
    HL[0] = np.eye(1)
    HR[-1] = np.eye(1)
    for i in range(Ns-1,0,-1):
        HR[i-1] = Sub.NCon([HR[i],S[i],np.conj(S[i])],[[1,2],[-1,3,1],[-2,3,2]])
    
    #sweep from left to right to calulate <O> on each site, use QR decomposition to keep the left-environment of T left-canonical,
    #the right-enviroment of T right-canonical
    for i in range(Ns):
        magnet = Sub.NCon([S[i],op,np.conj(S[i])],[[1,2,4],[2,3],[1,3,4]])
        per_site.append(magnet.item())
        if i < Ns-1:
            S[i],U = Sub.Mps_QR0P(S[i])
            HL[i+1] = Sub.NCon([HL[i],np.conj(S[i]),S[i]],[[1,2],[1,3,-1],[2,3,-2]])
            S[i+1] = np.tensordot(U,S[i+1],(1,0))
        else:
            continue
            
    return per_site

print("="*20+" 2-site "+ "="*20)

Mpo = GetMpo_IsingCluster_Obc(Dp)
T = InitMps(Ns,Dp,Ds)
HL,HR = InitH(Mpo,T)
T = OptT2(Mpo,HL,HR,T)

print("(2) Sigma_X per Site:{}".format(Get_magnet(T,pX)))
print("(2) Sigma_X average:{}".format(np.mean(Get_magnet(T,pX))))
print("(3) Sigma_Z per Site:{}".format(Get_magnet(T,pZ)))
print("(3) Sigma_Z average:{}".format(np.mean(Get_magnet(T,pZ))))

print("="*20+" 1-site "+ "="*20)

Mpo = GetMpo_IsingCluster_Obc(Dp)
T = InitMps(Ns,Dp,Ds)
HL,HR = InitH(Mpo,T)
T = OptT(Mpo,HL,HR,T)

print("(2) Sigma_X per Site:{}".format(Get_magnet(T,pX)))
print("(2) Sigma_X average:{}".format(np.mean(Get_magnet(T,pX))))
print("(3) Sigma_Z per Site:{}".format(Get_magnet(T,pZ)))
print("(3) Sigma_Z average:{}".format(np.mean(Get_magnet(T,pZ))))

print("="*20+" ED "+ "="*20)

#define many_body_operator
def many_body_operator(idx, oprts, size = Ns):
    "Tensor product of `orts` acting on indexes `idx`. Fills rest with Id."
    matrices = [eye if k not in idx else oprts[idx.index(k)] for k in range(size)]
    prod = matrices[0]
    for k in range(1, size):
        prod = np.kron(prod, matrices[k]) 
    return prod


Hamil = np.zeros([2**Ns,2**Ns])+0j
for t in range(Ns - 2):
    Hamil += -1*many_body_operator([t, t+1, t+2], [pZ,pY,pZ])
for b in range(Ns - 1):
    Hamil += -1*many_body_operator([b, b+1], [pZ,pZ])
for s in range(Ns):
    Hamil += -1*many_body_operator([s], [pX])
    
# Diagonalize the Hamiltonian
evals, evecs = np.linalg.eigh(Hamil)

# Find the 20 lowest eigenvalues
lowest_evals = np.sort(evals)[:1][0]/Ns

gs_vec = evecs[:,np.argsort(evals)[0]]

print("(1) Energy per Site:{}".format(lowest_evals))

magnet_x_list = []
for i in range(Ns):
    magnet_op = many_body_operator([i],[pX])
    eval_magnet = np.conjugate(gs_vec) @ magnet_op @ gs_vec
    magnet_x_list.append(eval_magnet)
print("(2) Sigma_X per Site:{}".format(magnet_x_list))
print("(2) Sigma_X average:{}".format(np.mean(magnet_x_list)))

magnet_z_list = []
for i in range(Ns):
    magnet_op = many_body_operator([i],[pZ])
    eval_magnet = np.conjugate(gs_vec) @ magnet_op @ gs_vec
    magnet_z_list.append(eval_magnet)
print("(3) Sigma_Z per Site:{}".format(magnet_z_list))
print("(3) Sigma_Z average:{}".format(np.mean(magnet_z_list)))

==================== 2-site ====================
(1) Energy per Site:[-1.38634154 -1.38634154 -1.38634154 -1.38634321 -1.38634516 -1.38634614
 -1.38634516 -1.38634321 -1.38634154 -1.38634154]
(1) Energy average:-1.3863430558206102
(2) Sigma_X per Site:[(0.7510064072984304+0j), (0.5943961871474106+0j), (0.5387317196589849+0j), (0.515362919204931+0j), (0.5058339931730402+0j), (0.5058479176372219+0j), (0.5153858598373573+0j), (0.5387046435386192+0j), (0.594307589803679+0j), (0.7509103090514877+0j)]
(2) Sigma_X average:(0.5810487546351163+0j)
(3) Sigma_Z per Site:[(-5.6911420021066306e-11-2.9387106750308507e-32j), (-4.045025425725157e-11+0j), (-5.026201677083009e-12+0j), (1.9218071578563922e-11+0j), (1.039279773351609e-12+0j), (6.895256587924337e-11+0j), (2.1197665844852054e-10+0j), (6.646394545839485e-11+0j), (-1.5016127230538245e-10+0j), (-2.819194322434271e-10+0j)]
(3) Sigma_Z average:(-1.6681805936613615e-11-2.938710675030851e-33j)
==================== 1-site ====================
(1) E